In [14]:
import keepa
import numpy as np
import requests
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
from sklearn import linear_model
import scipy.stats as stats

In [2]:
with open("api_key.txt") as key_file :
    api_key = key_file.readline().strip()
api = keepa.Keepa(api_key)

In [3]:
# do a product search to filter out books that may be good to look at
book_params = {
    'deltaPercent30_USED_gte': 20,
    'deltaPercent30_USED_lte': 99,
    'deltaPercent90_COUNT_USED_gte': -50,
    'deltaPercent90_COUNT_USED_lte': 99,
    'avg180_SALES_gte': 0,
    'avg180_SALES_lte': 7000000,
    'rootCategory': 283155
}
# don't include list price because there's no way to compare the list price with the used price at this point in the process
# have Mark go through the other parameters and see if there's anything he thinks would also be helpful to include

In [213]:
test_params = {
    "avg180_SALES_gte": 10000,
    "avg180_SALES_lte": 6000000,
    "current_USED_gte": 800,
    "current_USED_lte": 1200,
    "avg30_USED_gte": 1800,
    "avg30_USED_lte": 1000000000,
    "current_LISTPRICE_gte": 5000,
    "current_LISTPRICE_lte": 100000000,
    "avg180_LISTPRICE_gte": 5000,
    "avg180_LISTPRICE_lte": 100000000,
    "rootCategory": 283155,
    "perPage": 10
}

In [217]:
test_params = {
    "avg180_SALES_gte": 0,
    "avg180_SALES_lte": 4000000,
    "current_USED_gte": 3001,
    "current_USED_lte": 3500,
    "avg30_USED_gte": 3600,
    "avg30_USED_lte": 100000000,
    "current_LISTPRICE_gte": 9000,
    "current_LISTPRICE_lte": 10000000,
    "rootCategory": 283155,
    "perPage": 100
}

In [218]:
test = api.product_finder(test_params)

In [3]:
asins = ['1888799838',
'1111987254',
'0769857698',
'185617610X',
'0521860962',
'0387708820',
'0072852631',
'1943876339',
'1580017304',
'1269773186']
book_data = api.query(asins)

100%|██████████| 10/10 [00:03<00:00,  2.92it/s]


In [222]:
data = api.query(test[:10])

100%|██████████| 10/10 [00:09<00:00,  1.00it/s]


In [16]:
# now call the api to get the data on all these books
book_data = api.query(test[40:50])

100%|██████████| 10/10 [00:05<00:00,  1.94it/s]


In [225]:
allthedata = data + book_data

In [226]:
len(allthedata)

20

In [33]:
book_data[1]['asin']

'0155510088'

In [ ]:
# ------- THE METRICS --------
# Used Count % change from 90 average now to 90 day average a year ago (maybe have it's weight be linear based off the average
# of the averages)
# Used Count change in number (90 day average to 90 day average)
# Current Used Count
# Current List Price
# Current New Price
# Max Trade-in value over the past year
# % ROI from current used price + shipping (see below) compared to average 
# price it sold at (or during peak?) minus Amazon ($10+15%) fees
# Residuals of rolling average
# Max rolling average (amount made during peak)

In [148]:
# when it sold, how much it sold for, and the number of used offers at that time
two_yrs_ago = datetime.datetime.now() - relativedelta(years=2)
sell_prices = pd.DataFrame()
sell_dates = pd.DataFrame()
used_counts = pd.DataFrame()
for book in range(len(book_data)) :
    last_two_years = book_data[book]['data']['SALES_time'] > two_yrs_ago
    drop_dates_list = []
    sell_prices_list = []
    used_counts_list = []
    for day in np.where(last_two_years)[0] :
        if ((book_data[book]['data']['SALES'][day - 1] - 
            book_data[book]['data']['SALES'][day]) / 
            (book_data[book]['data']['SALES'][day - 1]) >= .04) :
            day_sold = book_data[book]['data']['SALES_time'][day]
            drop_dates_list.append(day_sold)
            try :
                day_sold_Used_index = np.where(book_data[book]['data']['USED_time'] == day_sold)[0][0] - 1 
            except IndexError :
                previous_days = np.where(book_data[book]['data']['USED_time'] < day_sold)
                day_sold_Used_index = previous_days[0][-1]
            sell_prices_list.append(book_data[book]['data']['USED'][day_sold_Used_index])
            try :
                day_sold_used_count = np.where(book_data[book]['data']['COUNT_USED_time'] == day_sold)[0][0] - 1
            except IndexError :
                previous_days_count = np.where(book_data[book]['data']['COUNT_USED_time'] < day_sold)
                day_sold_used_count = previous_days_count[0][-1]
            used_counts_list.append(book_data[book]['data']['COUNT_USED'][day_sold_used_count])
    drop_dates_df = pd.DataFrame({book_data[book]['asin']:drop_dates_list})
    sell_dates = pd.concat([sell_dates, drop_dates_df], ignore_index=True, axis=1)
    sell_prices_df = pd.DataFrame({book_data[book]['asin']:sell_prices_list})
    sell_prices = pd.concat([sell_prices, sell_prices_df], ignore_index=True, axis=1)
    used_counts_df = pd.DataFrame({book_data[book]['asin']:used_counts_list})
    used_counts = pd.concat([used_counts, used_counts_df], ignore_index=True, axis=1)

In [149]:
# change anytime it sold for less than $11.77 to counting it as not selling at all
for column in sell_prices.columns :
    for row in sell_prices.index :
        if(sell_prices[column][row] < 11.77) :
            sell_prices.iloc[row, column] = 0
            used_counts.iloc[row, column] = 0

In [156]:
# find the rolling 30 day sales total
sell_prices = sell_prices.fillna(0) #convert the NaNs to 0s so we can do math with them
used_counts = used_counts.fillna(0)
rolling_averages = pd.DataFrame()
num_sold = pd.DataFrame()
used_count_avg = pd.DataFrame()
for book in range(len(book_data)) :
    two_yrs_ago = datetime.datetime.now() - relativedelta(years=2)
    two_yrs_30days = two_yrs_ago + datetime.timedelta(days=30)
    mving_window = [two_yrs_ago, two_yrs_30days]
    rolling_average = []
    books_sold = []
    used_cnt = []
    for day in range(701) : # there's 701 days between the end of the moving window and today
        start = sell_dates[book] > mving_window[0]
        end = sell_dates[book] < mving_window[1]
        window = np.where(start & end)[0]
        rolling_average.append(np.sum(sell_prices[book][window]))
        books_sold.append(np.count_nonzero(sell_prices[book][window]))
        used_cnt.append(np.average(used_counts[book][window]) if len(used_counts[book][window]) != 0 else 0)
        for date in range(len(mving_window)) :
            mving_window[date] += datetime.timedelta(days=1)
    rolling_averages[book] = rolling_average
    num_sold[book] = books_sold
    used_count_avg[book] = used_cnt

In [6]:
# Variance of rolling sales totals (residuals)
regr = linear_model.LinearRegression()
now = datetime.datetime.now().date()
start_date = now + relativedelta(years=-2,days=30) 
x = np.arange(701).reshape(-1,1)
variance = []
for book in range(len(book_data)) :
    y = rolling_averages[book]
    regr.fit(x, y)
    score = regr.score(x,y)
    y_mean = np.average(y)
    sum_sqs = []
    for day in range(701) :
        sum_sqs.append((rolling_averages[book][day] - y_mean) ** 2)
    total_sum_sqs = np.sum(sum_sqs)
    residual_sum_sqs = total_sum_sqs * (1 - score)
    variance.append(residual_sum_sqs)

In [163]:
# Highest peak amount and date
peaks = pd.DataFrame()
peak_amount = []
peak_end_date = []
peak_num_sold = []
peak_used_avg = []
peak_avg_price = []
now = datetime.datetime.now().date()
start = now + relativedelta(years=-2,days=30)
date_range = pd.date_range(start, now)
for book in range(len(book_data)) :
    peak = np.amax(rolling_averages[book][335:]) # only look at past year
    peak_amount.append(peak)
    peak_time = np.where(rolling_averages[book] == peak)[0]
    peak_end_date.append(date_range[peak_time[-1]])
    peak_num_sold.append(num_sold[book][peak_time[-1]])
    peak_used_avg.append(used_count_avg[book][peak_time[-1]])
    peak_avg_price.append(peak_amount[book] / peak_num_sold[book])
peaks['Peak Amount'] = peak_amount  #### Do we even want to use the total amount it made during peak? 
# take this part out probably
# how can we calculate likelyhood to sell
# maybe take this and compare it with the average number of used offers during that time
peaks['Peak End Date'] = peak_end_date
peaks['Num Sold During Peak'] = peak_num_sold
peaks['Average # of Used Offers'] = peak_used_avg
peaks['Average Price'] = peak_avg_price

In [207]:
# USED COUNT % change and difference for 90 averages over a year
# compute a reimann sum for the step graph, with delta x as 12 hours
used_count_avgs = pd.DataFrame()
thirty_days_ago = datetime.datetime.now() - datetime.timedelta(days=90)
for year in range(2) :    
    used_count_avg = []
    for book in range(len(book_data)) :
        start = thirty_days_ago - (datetime.timedelta(days=365) * year)
        total_used_sum = []
        for twelve_hours in range(180) :
            time = start + (datetime.timedelta(hours=12) * twelve_hours)
            current = book_data[book]['data']['COUNT_USED_time'] < time
            last_value = book_data[book]['data']['COUNT_USED'][np.where(current)[0][-1]]
            current_used_count = 0 if(last_value == -1) else last_value # the data lists -1 where there are really 0 used offers
            total_used_sum.append(current_used_count)
        used_count_avg.append(np.average(total_used_sum))
    used_count_avgs['This year' if(not year) else 'Last year'] = used_count_avg

# now use the averages to compute our metrics
# also take out the current used count, list price, and new price
used_count_metrics = pd.DataFrame()
used_count_per_change = []
used_count_diff = []
current_used_count = []
current_list_price = []
current_new_price = []
current_used_price = []
for book in range(len(book_data)) :
    per_change = (used_count_avgs['This year'][book] - 
                  used_count_avgs['Last year'][book]) / used_count_avgs['Last year'][book]
    used_count_per_change.append(per_change)
    used_count_diff.append(used_count_avgs['This year'][book] - used_count_avgs['Last year'][book])
    cur_used = book_data[book]['data']['COUNT_USED'][-1]
    current_used_count.append(0 if cur_used == -1 else cur_used)
    if(np.isnan(book_data[book]['data']['NEW'][-1])) :
        current_new_price.append(book_data[book]['data']['NEW'][-2])
    else :
        current_new_price.append(book_data[book]['data']['NEW'][-1])
    try :
        book_data[book]['data']['LISTPRICE']
        if(np.isnan(book_data[book]['data']['LISTPRICE'][-1])) :
            current_list_price.append(current_new_price[book])
        else :
            current_list_price.append(book_data[book]['data']['LISTPRICE'][-1])
    except KeyError :
        current_list_price.append(current_new_price[book])
    current_used_price.append(book_data[book]['data']['USED'][-1])
    
used_count_metrics['USED_COUNT percent change'] = used_count_per_change
used_count_metrics['USED_COUNT difference'] = used_count_diff

In [192]:
# % ROI from current used price + shipping (see below) compared to average 
# price it sold at (or during peak?) minus Amazon ($10+15%) fees
# for current used price (how much we pay) add $3.99 for below $5.46 and $2 for between that and $10

# adjust each price it sold at for Amazon fees, counting anything less than $10 as not selling
# Sum up the Amazon-fee-adjusted prices it sold at
# add shipping to current used price
# calculate adjusted ROI


# create the function for deducting Amazon fees (15% + $10 flat fee)
Amazon_fees = lambda price : price - (price * .15) - 10

Amazon_fees(11.77) # below this it calculates lost money

# make function for adding shipping costs to cheaper books
def add_shipping(price) :
    if price < 5.46 :
        return(price + 3.99)
    elif price < 10 :
        return(price + 2)
    else :
        return price

# Calculate the total money the book made after Amazon fees in the past year
past_year = datetime.datetime.now() - relativedelta(years=1)
psuedo_roi = []
for book in sell_dates.columns :
    recently = np.where(sell_dates[book] > past_year)[0]
    total_revenue = np.sum(Amazon_fees(sell_prices[book][recently]))
    used_price = add_shipping(current_used_price[book])
    fake_roi = (total_revenue - used_price) / used_price
    psuedo_roi.append(fake_roi)

In [208]:
current_used_price

[45.03, 60.5, 8.54, 20.81, 63.47, nan, 127.28, 118.69, 179.36, 11.33]

In [10]:
# max trade-in value over the past year
max_trade_in = []
for book in range(len(book_data)) :
    try :
        last_365 = np.where(book_data[book]['data']['TRADE_IN_time'] > past_year)[0]
        trade_in_data = np.nan_to_num(book_data[book]['data']['TRADE_IN'][last_365])
        max_trade_in.append(np.amax(trade_in_data))
    except :
        max_trade_in.append(0)

In [193]:
metrics = pd.DataFrame({'Psuedo ROI': psuedo_roi, 
                        'USED COUNT % change': used_count_metrics['USED_COUNT percent change'],
                        'USED COUNT difference': used_count_metrics['USED_COUNT difference'],
                        'CURRENT USED COUNT': current_used_count,
                        'CURRENT LIST PRICE': current_list_price,
                        'CURRENT NEW PRICE': current_new_price,
                        'Max Trade-in Value': max_trade_in,
                        'Cyclicity': variance
                       })

In [194]:
metrics 

,Psuedo ROI,USED COUNT % change,USED COUNT difference,CURRENT USED COUNT,CURRENT LIST PRICE,CURRENT NEW PRICE,Max Trade-in Value,Cyclicity
0,29.899625,-0.409320,-5.416667,6,9.99,9.99,20.28,2.063728e+06
1,29.275429,-0.117171,-2.494444,14,109.99,81.88,22.07,1.635822e+06
2,12.055833,-0.329189,-8.677778,12,140.00,114.38,13.99,1.693074e+06
3,50.540350,0.586141,5.122222,10,193.00,661.14,48.70,1.841261e+07
4,135.150500,-0.334997,-9.694444,18,196.95,338.78,30.06,1.457532e+08
5,145.917293,12.989362,6.783333,0,160.83,160.83,0.00,7.634347e+06
6,18.472545,-0.638104,-7.777778,1,147.08,147.08,0.00,8.537788e+05
7,34.518312,-0.665167,-11.500000,3,87.95,48.00,12.96,9.102914e+05
8,22.206250,-0.540338,-11.200000,6,208.21,208.21,15.17,9.224949e+05
9,16.660000,-0.649985,-24.172222,12,95.00,67.84,0.00,1.420069e+06


In [ ]:
# now train the model
regr = linear_model.LinearRegression()
X = metrics
y = [#put the subjective 'scores' in here]
regr.fit(X,y)
regr.coef_

In [ ]:
# Questions for Mark ~ 
# what can knowing the used offer count when it sold be good for?
# If a book sold many times at a low value, that's not bad, but only slightly good right?
# We care more about how much a book can sell in a year than in a peak period right?
# So really we should calculate the sum total it made after Amazon fees in the past year then use that to compute the ROI
# compare to the current priced + shipping

In [ ]:
# Calculate the value/worth of it to us and expected value ~ compare it with current price
# when doing exp val if less than $10 just counts as not selling


In [164]:
peaks

,Peak Amount,Peak End Date,Num Sold During Peak,Average # of Used Offers,Average Price
0,192.10,2020-02-23,5,10.800000,38.420000
1,130.23,2019-06-16,3,22.333333,43.410000
2,136.78,2020-02-16,3,18.333333,45.593333
3,394.05,2020-02-03,3,17.333333,131.350000
4,1884.72,2020-02-12,38,17.447368,49.597895
5,372.35,2019-09-21,3,4.666667,124.116667
6,126.08,2020-05-03,1,1.000000,126.080000
7,205.12,2020-02-08,5,13.400000,41.024000
8,179.50,2019-11-24,2,13.500000,89.750000
9,138.00,2020-01-12,3,16.000000,46.000000


In [182]:
old_peak

,Peak Amount,End Date,Amount Sold,Avg # of Used Offers,Average Price
0,125.45,2019-02-26,1,13.000000,125.450000
1,186.46,2018-06-20,3,26.333333,62.153333
2,175.18,2019-02-13,3,25.333333,58.393333
3,350.00,2019-02-01,5,11.200000,70.000000
4,1734.22,2019-02-08,45,30.422222,38.538222
5,376.55,2018-09-23,3,2.000000,125.516667
6,111.20,2019-05-10,2,14.500000,55.600000
7,20.97,2019-03-07,1,16.000000,20.970000
8,0.00,2020-05-03,0,0.000000,0.000000
9,99.94,2019-01-17,2,13.000000,49.970000


In [54]:

# we can then create a graph of the likelyhood it will sell in that peak period at that given price
# ((probability it will sell) * (the price it would have sold at - Amazon fees) - (used price + shipping)) / (used pric+shipng)

# we set lambda to be the number of times it sold in that period times a discount factor   
lambdas = []
discount_factor = 0.8 # the demand for any book this year will descrease by an estimated 20%
for book in range(len(book_data)) :
    lambdas.append(peaks['Num Sold During Peak'][book] * discount_factor)

# given that a book sold, probability that it was our book that sold and not someone else's
def prob_our_book(used_count) :
    return .9 - (.13 * np.sqrt(used_count))

# given a certain lambda and given the probability that it was our book that sold, now find the probability that 
# the book sold during the peak period

In [181]:
# create the same peaks table for the peak 2 years ago
old_peak = pd.DataFrame()
new_peak_amt = []
new_peak_date = []
peak_num_sold = []
pk_used_avg = []
pk_avg_price = []
for book in range(len(book_data)) :
    date = peaks['Peak End Date'][book] - relativedelta(years=1)
    date_index = np.where(date_range == date)[0][0]
    new_peak_amt.append(np.amax(rolling_averages[book][date_index - 5:date_index + 5])) ######
    peak_end_index = np.where(rolling_averages[book] == new_peak_amt[book])[0][-1]
    new_peak_date.append(date_range[peak_end_index])
    peak_num_sold.append(num_sold[book][peak_end_index])
    pk_used_avg.append(used_count_avg[book][peak_end_index])
    pk_avg_price.append((new_peak_amt[book] / peak_num_sold[book]) if peak_num_sold[book] != 0 else 0)
old_peak['Peak Amount'] = new_peak_amt
old_peak['End Date'] = new_peak_date
old_peak['Amount Sold'] = peak_num_sold
old_peak['Avg # of Used Offers'] = pk_used_avg
old_peak['Average Price'] = pk_avg_price

In [209]:
book_value = []
roi = []
asin = []
rankings = pd.DataFrame()
for book in range(len(book_data)) :
    x = 0
    lam = lambdas[book]
    probs = pd.DataFrame()
    p = prob_our_book(current_used_count[book])
    while(stats.poisson.pmf(x, lam) >= .01) :
        dist = stats.binom(x, p)
        prob = []
        for trial in range(x + 1) :
            prob.append(dist.pmf(trial) * stats.poisson.pmf(x, p))
        probs = probs.append(pd.Series(prob), ignore_index=True)
        x += 1
    num_sold_probs = []
    for num in range(len(probs.columns)) :
        num_sold_probs.append(np.sum(probs[num]))
    book_value.append(Amazon_fees(peaks['Average Price'][book]) * np.sum(num_sold_probs[1:]))
    roi.append((book_value[book] - current_used_price[book]) / current_used_price[book])
    asin.append(book_data[book]['asin'])
rankings['ROI'] = roi
rankings['ASIN'] = asin
rankings = rankings.sort_values(by='ROI', ascending=False)

In [201]:
num_sold_probs

[0.7873935931489953,
 0.1882083004978718,
 0.022493293083435863,
 0.0017919415745080898,
 0.00010686370905145508,
 4.984169156777117e-06,
 1.5886246616950297e-07]

In [203]:
Amazon_fees(peaks['Average Price'][3]) * np.sum(num_sold_probs[1:])

21.610821819923483

In [204]:
book_data[3]['data']['USED']

array([74.91, 74.85, 74.91, ..., 20.86, 20.82, 20.81])

In [205]:
book_data[3]['asin']

'0769857698'